In [1]:
# import h5py
import config
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve, roc_curve

/home/subhankar/.conda/envs/pytorch/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
np.set_printoptions(threshold=np.inf, suppress=False, floatmode='fixed')
torch.set_printoptions(threshold=5000)

In [3]:
folder = '../Data/'
hdf_file_path = os.path.join(folder, 'Loan_Data.h5')
hdf_path_balanced = os.path.join(folder, 'Loan_Data_balanced.h5')

In [4]:
# hdf_file_path: Loan_Data
key_len = 1600
random.seed = 448
rand_ind = random.sample([i for i in range(int(key_len/2))], int(key_len/2))

train_lim = int(0.7*int(key_len/2))
test_lim = train_lim + int(0.25*int(key_len/2))
valid_lim = test_lim + int((1 - train_lim - test_lim)*int(key_len/2))
train, test, valid = rand_ind[:train_lim], rand_ind[train_lim: test_lim], rand_ind[test_lim:]

Data Generator

In [5]:
def DataCreatorSequence(t, file_path, max_len = 135):
    df = pd.read_hdf(file_path, 'X_' + str(t))
    lookup = pd.read_hdf(file_path, 'lookup_' + str(t)).values
    df.loc[(df.zero_balance == 1) == True, 'current_status'] = '46'
    df.current_status = df.current_status.astype('category', categories=config.unique_values['current_status'])
    # df.loc[df.current_status != '1', 'current_status'] = '0'
    # df.current_status = df.current_status.astype('category', categories=['0', '1'])
    df.state = df.state.astype('category', categories=config.unique_values['unique_states'])
    y = pd.get_dummies(df.current_status).values
    df = pd.get_dummies(df, columns=['state'])
    X = df[config.column_list['select_columns']].values
    X[np.isnan(X)]=-1
    
    batch_data = np.zeros((lookup.shape[0], max_len, X.shape[1])) # batch, seq, features
    batch_labels = -1*np.ones((lookup.shape[0], max_len, y.shape[1]))
    
    for ind in range(lookup.shape[0]):
        sent = X[ lookup[ind, 1]:lookup[ind, 2], :]
        label = y[ lookup[ind, 1]:lookup[ind, 2], :]
        curr_len = sent.shape[0]
        batch_data[ind, :curr_len, :] = sent
        batch_labels[ind, :curr_len, :] = label
        
    return (batch_data, batch_labels)

In [ ]:
X, y = DataCreatorSequence(1,hdf_file_path)

In [ ]:
X.shape, y.shape

In [ ]:
np.argmax(y, axis=2)[534,4], y[534,4,:]

In [6]:
class MyLSTMPredictor(nn.Module):

    def __init__(self, input_dim, hidden_dim, tagset_size, device):
        super(MyLSTMPredictor, self).__init__()
        self.hidden_dim = hidden_dim

        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)

        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.device = device
        self.hidden = 0 # self.init_hidden()

    def init_hidden(self, minibatch_size):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.FloatTensor(1, minibatch_size, self.hidden_dim).fill_(0).to(self.device),
                torch.FloatTensor(1, minibatch_size, self.hidden_dim).fill_(0).to(self.device))

    def forward(self, x):
        # seq_len = x.shape[1]
        batch_size = x.shape[0]
        # print(x.shape)
        self.hidden = self.init_hidden(batch_size)
        # x.view(seq_len, 1, -1)
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        # print(lstm_out.shape)
        lstm_out = lstm_out.contiguous().view(-1, lstm_out.shape[2])
        tag_space = self.hidden2tag(lstm_out)
        tag_scores = F.log_softmax(tag_space, dim=1)
        # print(tag_scores.shape)
        return tag_scores

In [ ]:
def DataCreator(t, file_path):
    df = pd.read_hdf(file_path, 'X_' + str(t))
    lookup = pd.read_hdf(file_path, 'lookup_' + str(t)).values
    df.current_status = df.current_status.astype('category', categories=config.unique_values['current_status'])
    # df.loc[df.current_status != '1', 'current_status'] = '0'
    # df.current_status = df.current_status.astype('category', categories=['0', '1'])
    df.state = df.state.astype('category', categories=config.unique_values['unique_states'])
    y = pd.get_dummies(df.current_status).values
    df = pd.get_dummies(df, columns=['state'])
    X = df[config.column_list['select_columns']].values
    X[np.isnan(X)]=-1
    return (X, y, lookup)

In [42]:
def GetAUCScore(y_actual, predicted_score, concerned_class, y_vector=False):
    ### y_actual: one hot encoding of all classes. np array of shape n x num_of_classes
    ### predicted_score: np array of shape n x num_of_classes containing probabilities
    ### PrintMatrix(predicted_score)
    # print(np.min(predicted_score), np.max(predicted_score))
    try:
#         print(predicted_score[:100, 0])
#         print('#################################')
#         print('########## class 0 #################')
#         print('#################################')
#         print(y_actual.shape)
#         print(predicted_score.shape)
        predicted_score = predicted_score[:, concerned_class]
        if y_vector == False:
            y_actual = y_actual[:, concerned_class]
        valid_index = (y_actual != -1)
        y_actual = y_actual[valid_index]
        
        predicted_score = predicted_score[valid_index]
        # print(roc_auc_score(y_actual, predicted_score), y_actual[y_actual == 1].shape, y_actual.shape)
#         print(y_actual.shape)
#         print('#################################')
#         print('########### actual class 1 ###############')
#         print('#################################')
#         print(predicted_score.shape)
#         print('#################################')
#         print('############ class 1 ############')
#         print('#################################')
        return roc_auc_score(y_actual, predicted_score)
    except Exception as e:
        print("Error", e)
        # print(predicted_score)
        return 0

Simple LSTM Model

In [ ]:
class LSTMPredictor(nn.Module):

    def __init__(self, input_dim, hidden_dim, tagset_size):
        super(LSTMPredictor, self).__init__()
        self.hidden_dim = hidden_dim

        self.lstm = nn.LSTM(input_dim, hidden_dim)

        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.FloatTensor(1, 1, self.hidden_dim).fill_(0).to(device),
                torch.FloatTensor(1, 1, self.hidden_dim).fill_(0).to(device))

    def forward(self, x):
        seq_len = x.shape[1]
        lstm_out, self.hidden = self.lstm(x.view(seq_len, 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out)
        tag_scores = F.softmax(tag_space, dim=2)
        return tag_scores

In [40]:
cuda = True
INPUT_DIM = 110
HIDDEN_DIM = 128
NUM_EPOCHS = 5
CONCERNED_CLASS = 1
############################# Set correct number of classes
NUM_CLASS = len(config.unique_values['current_status'])
# NUM_CLASS = 2
#############################
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MyLSTMPredictor(INPUT_DIM, HIDDEN_DIM, NUM_CLASS, device)
loss_function = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
model = model.to(device)

In [ ]:
XX, yy = DataCreatorSequence(4, hdf_file_path)

In [ ]:
print(yy.shape)
y_single = yy[1,36,:]
y_single

In [ ]:
np.argmax(yy[1,36,:])

In [20]:
u = np.array([[3,3,-1],[1,4,-1],[4,3,-1]])
v = np.array([[0.3,0.3,0.4],[0.1,0.4,0.1],[0.4,0.3,0.1]])

In [28]:
u[u >= 0], v[u >= 0]

(array([3, 3, 1, 4, 4, 3]),
 array([0.30000000, 0.30000000, 0.10000000, 0.40000000, 0.40000000,
        0.30000000]))

In [45]:
### NEW
training_acc = np.zeros(shape=(NUM_EPOCHS, 1))
test_acc = np.zeros(shape=(NUM_EPOCHS, 1))
sftmx_layer = nn.Softmax()
rolling_loss = 0
loss_count = 0
# Training the Model
for epoch in range(NUM_EPOCHS):
    correct = 0
    total = 0
    train_auc = 0
    count = 0
    st_time = time.time()
    for i, t in enumerate(train):
        XX, yy = DataCreatorSequence(t, hdf_file_path) # hdf_file_path
        pred_prob = np.array([])
        # print('XX.shape = ', XX.shape)
        for j in range(0, XX.shape[0], 100):
            # print('j = ', j)
            X, y_true= XX[j:(j + 100), :, :], yy[j:(j + 100), :, :]
            if X.shape[0] == 0:
                continue
            # print(X.shape, y.shape)
            inputs = Variable(torch.FloatTensor(X).to(device))
            y = np.argmax(y_true, axis=2)
            y_oh = Variable(torch.LongTensor(y.reshape(y.shape[0]*y.shape[1])).to(device))
            # print('y_oh.shape: ', y_oh.shape)
            optimizer.zero_grad()
            # Forward + Backward + Optimize
            outputs = model(inputs)
            # print('y_oh.shape: ', y_oh.shape)
            # print(y_oh)
            # print('outputs.shape: ', outputs.shape)

            loss = loss_function(outputs, y_oh)
            loss.backward()
            optimizer.step()
            # print(outputs.shape)
            # print(loss.data)
            #print(outputs.shape)
            y_true = y_true.reshape((y.shape[0]*y_true.shape[1], -1))
            #print(y_true.shape)
            auc = GetAUCScore(y_true, sftmx_layer(outputs).data.cpu().numpy(), CONCERNED_CLASS)
            print(auc)
            rolling_loss += loss.item()
            loss_count += 1
            if auc > 0:
                train_auc += auc
                count += 1
        if (i%100) == 0:
            print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f, Roc: %.4f' % (epoch+1, NUM_EPOCHS, \
                    i+1, len(train), float(rolling_loss/loss_count), (train_auc/count) if count > 0 else 0))

#         for item in lookup.itertuples():
#         #     model.zero_grad()
#             # Also, we need to clear out the hidden state of the LSTM,
#             # detaching it from its history on the last instance.
#             model.hidden = model.init_hidden()

#             xx = X[item[2] : item[3]].T
#             mx = np.max(xx, axis=0)
#             xx = xx / mx
#             inputs = Variable(torch.FloatTensor(xx).to(device))
#             y_oh = Variable(torch.LongTensor(np.argmax(y[item[2] : item[3]], axis=1)).to(device))

#             optimizer.zero_grad()
#             # Forward + Backward + Optimize
#             outputs = model(inputs)

#             loss = loss_function(outputs.view(y_oh.shape[0], NUM_CLASS), y_oh)
#             loss.backward()
#             optimizer.step()

#             if pred_prob.shape[0] == 0:
#                 pred_prob = outputs.data.cpu().numpy().reshape(y_oh.shape[0], NUM_CLASS)
#             else:
#                 pred_prob = np.vstack((pred_prob, outputs.data.cpu().numpy().reshape(y_oh.shape[0], NUM_CLASS)))
        
#         auc = GetAUCScore(y, pred_prob, CONCERNED_CLASS)
#         if auc > 0:
#             train_auc += auc
#             count += 1
        
#         if (i+1) % 100 == 0:
#             e_time = time.time()
#             # print(y_true.shape, outputs.data.shape)
#             print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f, Roc: %.4f, time: %.3f' % (epoch+1, NUM_EPOCHS, \
#                     i+1, len(train), loss.data[0], (train_auc/count) if count > 0 else 0, e_time - st_time))
#             st_time = time.time()
#     training_acc[epoch] = float(train_auc/count)
    
#     count = 0
#     test_auc = 0
#     print('####################### Testing #######################')
#     for i, t in enumerate(test):
#         X, y, lookup = DataCreator(t, hdf_file_path) # hdf_file_path
#         pred_prob = np.array([])
#         for item in lookup.itertuples():
#             inputs = Variable(torch.FloatTensor(X[item[2] : item[3]].T).to(device))
#             y_oh = Variable(torch.LongTensor(np.argmax(y[item[2] : item[3]], axis=1)).to(device))

#             outputs = model(inputs)

#             if pred_prob.shape[0] == 0:
#                 pred_prob = outputs.data.cpu().numpy().reshape(y_oh.shape[0], NUM_CLASS)
#             else:
#                 pred_prob = np.vstack((pred_prob, outputs.data.cpu().numpy().reshape(y_oh.shape[0], NUM_CLASS)))
        
#         auc = GetAUCScore(y, pred_prob, CONCERNED_CLASS)
#         if auc > 0:
#             test_auc += auc
#             count += 1
#     test_acc[epoch] = float(test_auc/count)

/home/subhankar/.conda/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """
/home/subhankar/.conda/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  
/home/subhankar/.conda/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.5233613445378151
0.6078899082568807
0.4226956963741104
0.4657299520219328
0.6972643097643098
Error Only one class present in y_true. ROC AUC score is not defined in that case.
0
0.3931466383079286
0.4671286425017768
0.48958972000817486
0.4777258380278877
0.5221573342493988
0.19047205831308572
0.6281915519472878
0.2932951482479784
0.6069150148097517
0.6617597004765147
0.6093426410703897
0.5145287030474841
0.6292186777623671
0.7180417665183156
0.5106457242582897
0.5146634854359281
0.5018172377985463
0.6211924541363794
0.5977716495498923
0.6854755341144039
0.5944657152801949
0.5453167115902965
0.8187116564417178
0.6584725901772976
0.45525198938992045
0.2840587360238015
0.36763715920565465
0.37855935716335576
0.44383220760753644
0.2755066987289591
0.5724029380902413
Epoch: [1/5], Step: [1/560], Loss: 0.0061, Roc: 0.5207
0.48858887817422053
0.19222000569962955
0.27469323598012374
0.3387706146926537
0.5398853609053799
0.25987939249553305
0.556563766760212
0.6215500016932507
0.5707620528771

0.4268524930747923
0.49458544839255497
0.5025978267323794
0.48668922729836434
0.4638261366885902
0.471687778158165
0.4423404570539649
0.4298759476223294
0.7072642452515829
0.4341755972864026
0.4126093813593814
0.3763690673747096
0.513081611416679
0.36089238845144356
0.9134244654749387
0.4044741252348788
0.865234375
0.5392277725726121
0.5313644214162349
0.41775040486823384
0.39691333597150774
0.501193596704737
Error Only one class present in y_true. ROC AUC score is not defined in that case.
0
0.399932042133877
0.4471324296141815
0.4459066551718309
0.4235310224856645
0.2937140804597701
0.4570005886970173
0.39507532235414267
0.4745906016375934
0.5386033134848301
0.5757470061620741
0.7137096774193549
Error Only one class present in y_true. ROC AUC score is not defined in that case.
0
0.42907801418439717
0.4326320550725917
0.40611130078525093
0.5421215128459587
0.4219258843346435
0.3760141093474427
0.5209350747681242
0.5459477340390341
0.39917127071823205
0.4147025659360304
0.7679124579124

0.27872844249843626
0.3903621131343904
0.8602311643835617
0.37887802804929877
0.4541411490230388
0.5051483420593368
0.4738942762535478
0.38179751704843495
0.48854197349042705
0.6352676659528909
0.3455850369725968
0.608261514224817
0.4843671416596815
0.5080575848732273
0.4245730206976407
0.496420756486402
0.470072528787199
0.6151055408970976
0.6545651233232366
0.5288283443866881
0.5371523453715233
0.5156160912258473
0.6776553002987535
0.5388805004314065
0.7098408554022131
0.14881720430107526
0.6351851851851852
0.5003743760399334
Error Only one class present in y_true. ROC AUC score is not defined in that case.
0
0.6984739676840216
0.43788812785388126
0.6778843179709249
0.0835829022597332
0.3232218232218232
0.47950481430536446
0.4578956343792633
0.5808538411333358
0.3883396632843943
0.5255066789497927
0.8985754985754986
0.6600460122699388
0.40271084337349394
0.4845532124394847
0.5601756747184246
0.3370133900860158
0.5165169111487098
0.5444660734149054
0.4612012717821532
0.442021441889372

0.7468676855293955
0.5578131545873481
0.37902303233679074
0.5284750613246116
0.7184867629362214
0.4883154335719968
0.14203750349846067
0.4099025974025974
0.319693094629156
0.2156862745098039
0.5210806385591487
0.47670666115018623
0.3260393873085339
Error Only one class present in y_true. ROC AUC score is not defined in that case.
0
0.36037796284433055
0.5166818181818182
0.3122989593188269
0.6373902132998746
0.44933692498963945
0.7141819735155917
0.29458967163885197
0.10365058670143418
0.7065772669220945
0.4240345110928513
0.6309327280231118
0.4688133095954325
0.7718962585034014
0.565965196276811
0.541531419059509
0.5105875753751228
0.9016393442622951
0.5533391293147861
0.5687709850071474
0.6330393902303327
0.39890648850186416
0.601149529326575
0.5949543676662321
0.35193337735710617
0.5878170637970792
0.45912119289340103
0.553026383859286
0.5619929453262786
0.37967986977753665
0.7106776694173542
0.24396181082448154
0.5154678307079144
0.5529809945326738
0.27112214498510423
Error Only one

...

0.6162648140580301
0.733486177565125
0.6019467620182757
Error Only one class present in y_true. ROC AUC score is not defined in that case.
0
0.5695609891636566
0.6258457374830853
0.6284227820372399
0.6153750311487665
0.48638253638253637
0.5942171548371414
0.6803684059633027
0.5702508455467868
0.599096993940354
0.5764620929317937
0.5592126424725543
0.5867195856058128
0.636434309406582
0.6040480135307721
0.5016606745420304
0.5443914328683297
0.6453849190682984
0.6229404309252218
0.1572362809212397
0.5037783375314862
0.606066894953478
0.5374235715791694
0.5831444205589214
0.5982784431137725
0.6139345867606737
0.5251288503883659
0.6056867136214442
0.5580828445747801
0.41202797202797203
0.38069139966273186
0.6175853587962963
0.5989486963835156
0.508151714816692
0.5118912985630655
0.493212055389628
0.6927966101694916
0.5280565693430657
0.7548910351659237
0.5664248592464269
0.6171159029649596
0.6295659483417393
0.20975719424460432
0.5934306569343065
0.5222171671216257
0.6867749419953597
0.736

0.6429306018347114
0.6390935595017228
0.5486853421796289
0.6527517783601647
0.5455999075287046
0.6281792684800984
0.5850765225122245
0.5635380509834659
0.6034436181032291
0.572356549981956
0.45007082152974504
0.6253252898036433
0.5655575905575906
0.5982874617737004
0.4514529914529914
0.5384197627137526
0.443928317595695
0.6315342960288809
0.4838518518518518
0.39297505502567864
0.382761231701161
0.6454741379310345
0.6331065575251622
0.6575931232091691
0.19989357928343382
0.7488981073373088
0.36871704745166956
0.6473509933774835
0.5774431261197309
0.553991799880223
0.5260581174984207
0.6071622751986616
0.5978911461107307
0.5238466334164589
0.5554463194868662
0.5782771535580524
0.5917402079303675
0.43633814102564095
0.5860521981113738
0.47867395762132603
0.6309987029831388
0.4354940270581462
0.4535340314136125
0.5208054812834224
0.6501305483028721
0.6515448851774531
0.7001608279141319
0.6597980908052664
0.4980352303523035
0.639575411913815
0.406260671599317
0.7199612403100775
0.6276330974

0.5293909661451428
0.5595381310418904
0.5659927213910231
0.63687374749499
0.6628142741281428
0.6099579884364212
0.7388888888888889
0.5789449499126919
0.6437229010642491
0.36955653216140627
0.5466403981750311
0.5961433608374831
0.7129228480154042
0.5048751402918069
0.5352709336354197
0.8322140822140822
0.6952631376175066
0.5533987603305784
0.5455112219451371
0.3593326802282026
0.5895792241946087
Error Only one class present in y_true. ROC AUC score is not defined in that case.
0
0.3739922605611093
0.48585485854858546
0.6269289027738332
0.5383215369059656
0.3197539075490522
0.4191496062992126
0.6310562507045429
0.6349572086899277
0.6252246469833119
0.6585450748346676
0.5415446360776658
0.5850653556288272
0.6238877481177276
0.6052631578947368
0.6164943457189015
0.5793731041456016
0.4483166612885877
0.5191264991039838
0.6195948136142626
0.52183988970049
0.6009754177237338
0.60357025337282
0.6506442021803766
0.5534979423868314
0.140987408880053
0.6327081192189106
0.5490933971946631
0.344496

0.7112674362674363
0.630106143454487
0.5601357904946654
0.5307546204982102
0.46639027086788276
0.3662364563416189
0.6110583768391077
0.5901976935749588
0.5967644444444445
0.6290485670047714
0.5377068946618437
0.6752876240160232
0.5952238550922762
0.6190052259168478
0.553846966814627
0.5335715095837587
0.6121100750235398
Error Only one class present in y_true. ROC AUC score is not defined in that case.
0
0.6173178226514486
0.644421052631579
0.5583981001727115
0.4533993696533093
0.7076234038498189
0.5930306359087025
0.6308397821725422
0.5654072719886248
0.6314144736842106
0.6293307555443477
0.6198882024124743
0.5541976007717797
0.5648150157317999
0.6475071858746236
0.6719690101376411
0.6286826347305389
0.6208401847936732
0.6301041960011264
0.6127856046238599
0.6145601336302896
0.5658541194255481
0.6468217761557177
0.5438645429440152
0.60943538647343
0.42347535505430245
0.6185991812697501
0.6206643151087595
0.6579697498545666
0.6326447954637505
0.3853448275862069
0.6595049845307666
0.5432

KeyboardInterrupt: 

In [ ]:
training_acc = np.zeros(shape=(NUM_EPOCHS, 1))
test_acc = np.zeros(shape=(NUM_EPOCHS, 1))

# Training the Model
for epoch in range(NUM_EPOCHS):
    correct = 0
    total = 0
    train_auc = 0
    count = 0
    st_time = time.time()
    for i, t in enumerate(train):
        X, y, lookup = DataCreator(t, hdf_file_path) # hdf_file_path
        pred_prob = np.array([])
        for item in lookup.itertuples():
        #     model.zero_grad()
            # Also, we need to clear out the hidden state of the LSTM,
            # detaching it from its history on the last instance.
            model.hidden = model.init_hidden()

            xx = X[item[2] : item[3]].T
            mx = np.max(xx, axis=0)
            xx = xx / mx
            inputs = Variable(torch.FloatTensor(xx).to(device))
            y_oh = Variable(torch.LongTensor(np.argmax(y[item[2] : item[3]], axis=1)).to(device))

            optimizer.zero_grad()
            # Forward + Backward + Optimize
            outputs = model(inputs)

            loss = loss_function(outputs.view(y_oh.shape[0], NUM_CLASS), y_oh)
            loss.backward()
            optimizer.step()

            if pred_prob.shape[0] == 0:
                pred_prob = outputs.data.cpu().numpy().reshape(y_oh.shape[0], NUM_CLASS)
            else:
                pred_prob = np.vstack((pred_prob, outputs.data.cpu().numpy().reshape(y_oh.shape[0], NUM_CLASS)))
        
        auc = GetAUCScore(y, pred_prob, CONCERNED_CLASS)
        if auc > 0:
            train_auc += auc
            count += 1
        
        if (i+1) % 100 == 0:
            e_time = time.time()
            # print(y_true.shape, outputs.data.shape)
            print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f, Roc: %.4f, time: %.3f' % (epoch+1, NUM_EPOCHS, \
                    i+1, len(train), loss.data[0], (train_auc/count) if count > 0 else 0, e_time - st_time))
            st_time = time.time()
    training_acc[epoch] = float(train_auc/count)
    
    count = 0
    test_auc = 0
    print('####################### Testing #######################')
    for i, t in enumerate(test):
        X, y, lookup = DataCreator(t, hdf_file_path) # hdf_file_path
        pred_prob = np.array([])
        for item in lookup.itertuples():
            inputs = Variable(torch.FloatTensor(X[item[2] : item[3]].T).to(device))
            y_oh = Variable(torch.LongTensor(np.argmax(y[item[2] : item[3]], axis=1)).to(device))

            outputs = model(inputs)

            if pred_prob.shape[0] == 0:
                pred_prob = outputs.data.cpu().numpy().reshape(y_oh.shape[0], NUM_CLASS)
            else:
                pred_prob = np.vstack((pred_prob, outputs.data.cpu().numpy().reshape(y_oh.shape[0], NUM_CLASS)))
        
        auc = GetAUCScore(y, pred_prob, CONCERNED_CLASS)
        if auc > 0:
            test_auc += auc
            count += 1
    test_acc[epoch] = float(test_auc/count)

In [ ]:
X, y, lookup = DataCreator(1, hdf_file_path) # hdf_file_path

In [ ]:
pred_prob.shape

In [ ]:
y.shape

In [ ]:
GetAUCScore(y, pred_prob, 42)

In [ ]:
train_auc/count